In [3]:
import qubx  # type: ignore
from qubx.core.basics import DataType

%qubxd

%load_ext autoreload
%autoreload 2

# - - - - - - - - - -
from qubx.data.transformers import PandasFrame, TimestampedList, OHLCVSeries
from qubx.data.storages.csv import CsvStorage

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Basic CSV storage tests

In [2]:
r = CsvStorage("~/devs/Qubx/tests/data/storages/csv")

print(r.get_exchanges())
print(r.get_market_types("BINANCE.UM"))

['BINANCE.UM']
['SWAP']


## Reader

In [27]:
bnc_swap = r["BINANCE.UM", "SWAP"]

In [28]:
bnc_swap.get_data_types("BTCUSDT")

['ohlc(1h)', quote]

In [29]:
bnc_swap.get_data_id("ohlc(1h)")

['ETHUSDT', 'BCHUSDT', 'BTCUSDT', 'LTCUSDT', 'AAVEUSDT']

In [6]:
print(bnc_swap.get_time_range("BTCUSDT", DataType.OHLC["1h"]))
print(bnc_swap.get_time_range("BTCUSDT", DataType.QUOTE))

(numpy.datetime64('2023-06-01T00:00:00'), numpy.datetime64('2023-08-01T00:00:00'))
(numpy.datetime64('2017-08-24T13:01:12'), numpy.datetime64('2017-08-24T13:01:59'))


In [19]:
ds = bnc_swap.read("BTCUSDT", DataType.OHLC['1h'], chunksize=0) # type: ignore

In [35]:
print(bnc_swap.read("BTCUSDT", DataType.QUOTE).raw[:10])

[[Timestamp('2017-08-24 13:01:12') 10.1 10.0 200 100]
 [Timestamp('2017-08-24 13:01:13') 10.1 10.01 20000 100]
 [Timestamp('2017-08-24 13:01:14') 10.13 10.03 200 100]
 [Timestamp('2017-08-24 13:01:15') 10.12 10.01 20000 100]
 [Timestamp('2017-08-24 13:01:16') 10.11 10.1 200 100]
 [Timestamp('2017-08-24 13:01:17') 10.19 10.16 200 100]
 [Timestamp('2017-08-24 13:01:18') 10.19 10.16 200 10000]
 [Timestamp('2017-08-24 13:01:19') 10.19 10.15 200 100]
 [Timestamp('2017-08-24 13:01:20') 10.18 10.15 20000 10000]
 [Timestamp('2017-08-24 13:01:21') 10.18 10.15 200 10000]]


In [32]:
print(bnc_swap.read("ETHUSDT", DataType.TRADE).raw[:10])

[[3920436870 3054.91 0.1 305.491 1713571200068 True]
 [3920436871 3054.91 0.144 439.90704 1713571200199 True]
 [3920436872 3054.91 0.328 1002.01048 1713571200199 True]
 [3920436873 3054.91 0.328 1002.01048 1713571200202 True]
 [3920436874 3054.91 0.037 113.03167 1713571200205 True]
 [3920436875 3054.92 0.007 21.38444 1713571200209 False]
 [3920436876 3054.91 0.328 1002.01048 1713571200225 True]
 [3920436877 3054.91 0.917 2801.35247 1713571200279 True]
 [3920436878 3054.92 0.027 82.48284 1713571200297 False]
 [3920436879 3054.92 0.174 531.55608 1713571200297 False]]


In [33]:
print(bnc_swap.read("ETHUSDT", DataType.OHLC["1Min"]).raw[:10])

ValueError: Can't find any csv data for 'ETHUSDT' of 'ohlc(1Min)' in /home/quant0/devs/Qubx/tests/data/storages/csv/BINANCE.UM/SWAP !

## Conversion

In [23]:
r = CsvStorage("~/devs/Qubx/tests/data/storages/csv")
bnc_swap = r["BINANCE.UM", "SWAP"]

raw1 = bnc_swap.read("BTCUSDT", DataType.QUOTE, None, None)
raw2 = bnc_swap.read("BTCUSDT", DataType.OHLC["1h"], None, None)
raw3 = bnc_swap.read("ETHUSDT", DataType.OHLC["1h"], None, None)
raw4 = bnc_swap.read("ETHUSDT", DataType.TRADE, None, None)

In [24]:
raw4.transform(PandasFrame())

,id,price,qty,quote_qty,is_buyer_maker
time,,,,,
1713571200068,3920436870,3054.91,0.1,305.491,True
1713571200199,3920436871,3054.91,0.144,439.90704,True
1713571200199,3920436872,3054.91,0.328,1002.01048,True
1713571200202,3920436873,3054.91,0.328,1002.01048,True
1713571200205,3920436874,3054.91,0.037,113.03167,True
...,...,...,...,...,...
1713571229272,3920438865,3056.52,0.01,30.5652,True
1713571229272,3920438866,3056.52,0.005,15.2826,True
1713571229315,3920438867,3056.52,4.674,14286.17448,False


In [25]:
raw4.transform(TimestampedList())

[[1970-01-01T00:28:33.571200068]	3054.91000 (0.10) buy 3920436870,
 [1970-01-01T00:28:33.571200199]	3054.91000 (0.14) buy 3920436871,
 [1970-01-01T00:28:33.571200199]	3054.91000 (0.33) buy 3920436872,
 [1970-01-01T00:28:33.571200202]	3054.91000 (0.33) buy 3920436873,
 [1970-01-01T00:28:33.571200205]	3054.91000 (0.04) buy 3920436874,
 [1970-01-01T00:28:33.571200209]	3054.92000 (0.01) ??? 3920436875,
 [1970-01-01T00:28:33.571200225]	3054.91000 (0.33) buy 3920436876,
 [1970-01-01T00:28:33.571200279]	3054.91000 (0.92) buy 3920436877,
 [1970-01-01T00:28:33.571200297]	3054.92000 (0.03) ??? 3920436878,
 [1970-01-01T00:28:33.571200297]	3054.92000 (0.17) ??? 3920436879,
 [1970-01-01T00:28:33.571200297]	3054.92000 (0.03) ??? 3920436880,
 [1970-01-01T00:28:33.571200297]	3054.92000 (0.03) ??? 3920436881,
 [1970-01-01T00:28:33.571200297]	3054.92000 (0.09) ??? 3920436882,
 [1970-01-01T00:28:33.571200325]	3054.92000 (0.02) ??? 3920436883,
 [1970-01-01T00:28:33.571200335]	3054.92000 (0.01) ??? 3920436

In [31]:
raw2.transform(PandasFrame(True))

,,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume
timestamp,symbol,,,,,,,,,
2023-06-01 00:00:00,BTCUSDT,27201.1,27347.9,27008.1,27059.8,33639.568,914441790.0,256082,15501.039,421503008.0
2023-06-01 01:00:00,BTCUSDT,27059.8,27155.7,27045.7,27065.5,17636.072,478001888.0,141280,8794.62,238385216.0
2023-06-01 02:00:00,BTCUSDT,27065.5,27106.5,26621.3,26705.1,42682.676,1144205950.0,268982,17221.14,461549856.0
2023-06-01 03:00:00,BTCUSDT,26705.0,26829.0,26600.0,26768.6,27103.262,724444350.0,219415,14888.465,398015552.0
2023-06-01 04:00:00,BTCUSDT,26768.6,26813.6,26742.0,26797.2,9880.983,264626432.0,94780,5605.699,150137440.0
...,...,...,...,...,...,...,...,...,...,...
2023-07-31 20:00:00,BTCUSDT,29146.4,29249.0,29102.3,29194.5,13962.28,407463040.0,127312,7628.219,222628592.0
2023-07-31 21:00:00,BTCUSDT,29194.5,29285.3,29144.8,29230.6,10977.811,320717216.0,86779,5551.999,162209840.0
2023-07-31 22:00:00,BTCUSDT,29230.6,29232.1,29186.0,29199.6,4243.307,123939200.0,43890,1955.245,57107704.0


In [46]:
names = ["o", "t", "b"]
dict(zip(names, range(len(names))))

{'o': 0, 't': 1, 'b': 2}

In [45]:
raw2.transform(OHLCVSeries())

                        open     high      low    close     volume  \
timestamp                                                            
2023-06-01 00:00:00  27201.1  27347.9  27008.1  27059.8  33639.568   
2023-06-01 01:00:00  27059.8  27155.7  27045.7  27065.5  17636.072   
2023-06-01 02:00:00  27065.5  27106.5  26621.3  26705.1  42682.676   
2023-06-01 03:00:00  26705.0  26829.0  26600.0  26768.6  27103.262   
2023-06-01 04:00:00  26768.6  26813.6  26742.0  26797.2   9880.983   
...                      ...      ...      ...      ...        ...   
2023-07-31 20:00:00  29146.4  29249.0  29102.3  29194.5  13962.280   
2023-07-31 21:00:00  29194.5  29285.3  29144.8  29230.6  10977.811   
2023-07-31 22:00:00  29230.6  29232.1  29186.0  29199.6   4243.307   
2023-07-31 23:00:00  29199.5  29229.8  29173.4  29220.8   3765.949   
2023-08-01 00:00:00  29220.8  29277.0  29200.9  29272.3   4934.267   

                     bought_volume  volume_quote  bought_volume_quote  \
timestamp       

In [32]:
srows(raw2.transform(PandasFrame(True)), raw3.transform(PandasFrame(True)))

open     high      low    close      volume  \
timestamp           symbol                                                    
2023-06-01 00:00:00 BTCUSDT  27201.1  27347.9  27008.1  27059.8   33639.568   
                    ETHUSDT  1872.85  1889.61   1867.2  1870.65  286414.624   
2023-06-01 01:00:00 BTCUSDT  27059.8  27155.7  27045.7  27065.5   17636.072   
                    ETHUSDT  1870.65  1879.05  1870.39  1873.32  112125.784   
2023-06-01 02:00:00 BTCUSDT  27065.5  27106.5  26621.3  26705.1   42682.676   
...                              ...      ...      ...      ...         ...   
2023-07-31 22:00:00 ETHUSDT  1856.38   1857.0  1854.17   1855.5   25662.458   
2023-07-31 23:00:00 BTCUSDT  29199.5  29229.8  29173.4  29220.8    3765.949   
                    ETHUSDT  1855.49   1857.3  1854.16  1855.39   23961.846   
2023-08-01 00:00:00 BTCUSDT  29220.8  29277.0  29200.9  29272.3    4934.267   
                    ETHUSDT   1855.4  1861.74  1854.27  1860.99   57546.532   

                             quote_volume   count taker_buy_volume  \
timestamp           symbol                                           
2023-06-01 00:00:00 BTCUSDT   914441790.0  256082        15501.039   
                    ETHUSDT   538209090.0  208626       147333.712   
2023-06-01 01:00:00 BTCUSDT   478001888.0  141280          8794.62   
                    ETHUSDT   210286992.0   99710          56460.7   
2023-06-01 02:00:00 BTCUSDT  1144205950.0  268982         17221.14   
...                                   ...     ...              ...   
2023-07-31 22:00:00 ETHUSDT    47610984.0   32540        12914.089   
2023-07-31 23:00:00 BTCUSDT   109981072.0   42779         1826.605   
                    ETHUSDT    44462552.0   28068        10439.992   
2023-08-01 00:00:00 BTCUSDT   144272672.0   49285         2739.515   
                    ETHUSDT   106932632.0   41037        34820.892   

                            taker_buy_quote_volume  
timestamp           symbol                          
2023-06-01 00:00:00 BTCUSDT            421503008.0  
                    ETHUSDT            276909632.0  
2023-06-01 01:00:00 BTCUSDT            238385216.0  
                    ETHUSDT            105882736.0  
2023-06-01 02:00:00 BTCUSDT            461549856.0  
...                                            ...  
2023-07-31 22:00:00 ETHUSDT             23957768.0  
2023-07-31 23:00:00 BTCUSDT             53346232.0  
                    ETHUSDT             19372540.0  
2023-08-01 00:00:00 BTCUSDT             80101792.0  
                    ETHUSDT             64708432.0  

[2930 rows x 9 columns]